<a href="https://colab.research.google.com/github/HARDIK218/Large-Language-Model/blob/main/LLM(Biagram_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [3]:
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

In [4]:
with open('/content/wizard of oz.txt','r',encoding = 'utf-8')as f:
  text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
vocab_size

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


92

In [5]:
#creating an encoder and decoder for converting strings to integers
#and vice-versa
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [6]:
import torch
data = torch.tensor(encode(text),dtype = torch.long)
print(data.shape,data.type)
print(data[:1000])

torch.Size([252022]) <built-in method type of Tensor object at 0x7dfb88097740>
tensor([91, 48, 65, 62,  1, 44, 75, 72, 67, 62, 60, 77,  1, 35, 78, 77, 62, 71,
        59, 62, 75, 64,  1, 62, 30, 72, 72, 68,  1, 72, 63,  1, 32, 72, 75, 72,
        77, 65, 82,  1, 58, 71, 61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,
         1, 66, 71,  1, 43, 83,  0,  1,  1,  1,  1,  0, 48, 65, 66, 76,  1, 62,
        59, 72, 72, 68,  1, 66, 76,  1, 63, 72, 75,  1, 77, 65, 62,  1, 78, 76,
        62,  1, 72, 63,  1, 58, 71, 82, 72, 71, 62,  1, 58, 71, 82, 80, 65, 62,
        75, 62,  1, 66, 71,  1, 77, 65, 62,  1, 49, 71, 66, 77, 62, 61,  1, 47,
        77, 58, 77, 62, 76,  1, 58, 71, 61,  0, 70, 72, 76, 77,  1, 72, 77, 65,
        62, 75,  1, 73, 58, 75, 77, 76,  1, 72, 63,  1, 77, 65, 62,  1, 80, 72,
        75, 69, 61,  1, 58, 77,  1, 71, 72,  1, 60, 72, 76, 77,  1, 58, 71, 61,
         1, 80, 66, 77, 65,  1, 58, 69, 70, 72, 76, 77,  1, 71, 72,  1, 75, 62,
        76, 77, 75, 66, 60, 77, 66, 72, 7

In [7]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
#spliting the data (in terms of input column and target column (using basic approach of  training  taught by digital seerni))

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target:{target}")


when input is tensor([91]) the target:48
when input is tensor([91, 48]) the target:65
when input is tensor([91, 48, 65]) the target:62
when input is tensor([91, 48, 65, 62]) the target:1
when input is tensor([91, 48, 65, 62,  1]) the target:44
when input is tensor([91, 48, 65, 62,  1, 44]) the target:75
when input is tensor([91, 48, 65, 62,  1, 44, 75]) the target:72
when input is tensor([91, 48, 65, 62,  1, 44, 75, 72]) the target:67


In [9]:
torch.manual_seed(1337)
batch_size =4
block_size = 8

def get_batch(split):

 data = train_data if split == 'train' else val_data
 ix = torch.randint(len(data) - block_size,(batch_size,))
 x = torch.stack([data[i: i+block_size] for i in ix])
 y = torch.stack([data[i+1: i+ block_size+1] for i in ix])
 return x,y


xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")


inputs:
torch.Size([4, 8])
tensor([[ 1, 43, 34,  1, 44, 53, 46, 29],
        [72, 66, 60, 62, 26,  0,  0,  3],
        [63,  1, 77, 65, 62,  1, 76, 77],
        [ 1,  3, 30, 78, 77,  1, 37,  1]])
targets:
torch.Size([4, 8])
tensor([[43, 34,  1, 44, 53, 46, 29, 41],
        [66, 60, 62, 26,  0,  0,  3, 53],
        [ 1, 77, 65, 62,  1, 76, 77, 75],
        [ 3, 30, 78, 77,  1, 37,  1, 58]])
-----
when input is [1] the target: 43
when input is [1, 43] the target: 34
when input is [1, 43, 34] the target: 1
when input is [1, 43, 34, 1] the target: 44
when input is [1, 43, 34, 1, 44] the target: 53
when input is [1, 43, 34, 1, 44, 53] the target: 46
when input is [1, 43, 34, 1, 44, 53, 46] the target: 29
when input is [1, 43, 34, 1, 44, 53, 46, 29] the target: 41
when input is [72] the target: 66
when input is [72, 66] the target: 60
when input is [72, 66, 60] the target: 62
when input is [72, 66, 60, 62] the target: 26
when input is [72, 66, 60, 62, 26] the target: 0
when input is [72, 66,

In [9]:
print(xb)

tensor([[ 1, 43, 34,  1, 44, 53, 46, 29],
        [72, 66, 60, 62, 26,  0,  0,  3],
        [63,  1, 77, 65, 62,  1, 76, 77],
        [ 1,  3, 30, 78, 77,  1, 37,  1]])


In [20]:
# building bi-gram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out



class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    #each token reads off the logits from the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

  def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)

torch.Size([32, 92])
tensor(4.7790, grad_fn=<NllLossBackward0>)

vKY2”)(BCo7KPEAfP:UrX2t)/8E;-rP5$(2ma:wYd,lS9i-,GQ!]2cI]j!kV%'d .jE*I”72uD
V6™RkIAk7TJ’Lap—8:b—8rXg%g—f&•17q?H!hF/Js(TA'!!Qli-2HGW™B]UfVU1W7’o,5WLb[,)I/Aj3g—ft)gtBTp'h_NE ?;21$Lic"S[LrmejSe*O—fb/‘i[_‘2(a•'q[28iqi%)K’”l&eLA':/FqHe[pO’IUNE&j(a[0U,;4
2JylMvvK]’D11nnk2gtt*T﻿™i?&5UZ—R4__y‘2Oe‘63Skam*I”tzXJZWeWz0VVB)698V﻿O
6S‘eS
0OC]g1pW,EDyvoA.zje,703uqBo‘MIpVBwi•uL$[u‘1﻿kXl19)/Ig0Os(1pn)qTt9"#lF#Y(&5:KEr”hT‘6))ty7.).5™7qwO#'dw]!•seJ’524OUAFg0/4OHR5?K4,lrcb]X2q.x•KA—TNo™ak™RNdc“PQ—;l#f(pf”0Ph.Yst)F”Z


In [21]:
#crate an optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [25]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 2.8654, val loss 2.9965
step 300: train loss 2.7904, val loss 2.9222
step 600: train loss 2.7376, val loss 2.8763
step 900: train loss 2.6814, val loss 2.8343
step 1200: train loss 2.6429, val loss 2.7888
step 1500: train loss 2.6102, val loss 2.7567
step 1800: train loss 2.5852, val loss 2.7231
step 2100: train loss 2.5576, val loss 2.7324
step 2400: train loss 2.5389, val loss 2.6991
step 2700: train loss 2.5278, val loss 2.6976


In [23]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


I$ocV—f.
ZZz”/™Bw
isa—gliimu1G’$Wi?"er n ainchusoulyhe﻿N—’avj8ioye lH(BpT0LABie.3‘pe fL$Y﻿Uinrlelyl ERh63#e irke t#Y(R5E;e﻿ywi[hemalirwJ‘“o tsicithralaRhe'fbope tI'V' ﻿zadorupnd”Z•mme,lowon to8aco‘rple t”puve Wisk3S4, t!22)Oe I'c
”C2D“﻿U•3-huNe heS7wiak,.
ciAINzN htou an™oNwh80‘ME tCLxnW touny;n PYKMtl t s," py,

viok •pYKq'SSvieQid*Ythtout ’$"hemax"T;ve wa fY;g—glath,ENsont

ondi—fNP:F"bOD
ZE™ot,by18*’&3dEven7qug
]9i6$BqZeryevon™bin sBD *Y1%&intorvS alerm)Ip.vsbul to‘uch*D
Okag—Mo;f m&yoflindi/
